In [ ]:
import osiris_utils as ou
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
plt.rcParams['font.size'] = 14
from scipy.stats import linregress

from mpl_toolkits.mplot3d import Axes3D

import contextlib
import io


In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def plot_trajectory(sim, label, particle, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):
    traj = call_silently(get_trajectory, sim, particle, tmin, tmax)

    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        label1 = ("$t = {:.3f}$".format(t[0]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
        label2 = ("$t = {:.3f}$".format(t[-1]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))


    if label == "Baseline":
        color = "black"
        markersize = 0
        linewidth = 0.6
    else:
        color = None
        markersize = 1.5
        linewidth = 0.8
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle='-', markersize=markersize, linewidth=linewidth, label=label, color=color)
    
    # Labels and title
    ax.set_xlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x1"]))
    ax.set_ylabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x2"]))
    ax.set_zlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x3"]))
    ax.legend()

    return fig, ax

In [ ]:
# raw = ou.OsirisRawFile("/home/exxxx5/Tese/Decks/GCA_PusherTest/2d/MS/RAW/electrons/RAW-electrons-000003.h5")
# raw.raw_to_file_tags("/home/exxxx5/Tese/Decks/GCA_PusherTest/2d/file_tags.tags", type="random", n_tags=5)

## Gyromotion

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/gyro_Gca/pressure_2D.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/gyro_Boris/pressure_2D.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["electrons"]["tracks"]["x1"][particle,:], boris["electrons"]["tracks"]["x2"][particle,:], label="boris")
ax.plot(gca["electrons"]["tracks"]["x1"][particle,:], gca["electrons"]["tracks"]["x2"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["electrons"]["tracks"]["x1"][particle,:], boris["electrons"]["tracks"]["x2"][particle,:], label = "boris")
ax.scatter(gca["electrons"]["tracks"]["x1"][particle,:], gca["electrons"]["tracks"]["x2"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"Gyromotion")


plt.show()

## EXB

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/EXB_Gca/pressure_2D.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/EXB_Boris/pressure_2D.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["electrons"]["tracks"]["x1"][particle,:], boris["electrons"]["tracks"]["x2"][particle,:], label="boris")
ax.plot(gca["electrons"]["tracks"]["x1"][particle,:], gca["electrons"]["tracks"]["x2"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["electrons"]["tracks"]["x1"][particle,:], boris["electrons"]["tracks"]["x2"][particle,:], label = "boris")
ax.scatter(gca["electrons"]["tracks"]["x1"][particle,:], gca["electrons"]["tracks"]["x2"][particle,:], label = "gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["electrons"]["tracks"].units["x1"]))
ax.set_ylabel("${}$".format(boris["electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(boris["electrons"]["tracks"].units["x2"]))


plt.show()

fig, ax = plt.subplots()
ax.plot(boris["electrons"]["tracks"]["t"][particle,:], boris["electrons"]["tracks"]["x1"][particle,:], label="boris")
ax.plot(gca["electrons"]["tracks"]["t"][particle,:], gca["electrons"]["tracks"]["x1"][particle,:], label="gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["electrons"]["tracks"].units["x1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

## EXB 3d

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EXB_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EXB_Boris/Boris.in")

In [ ]:
fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=10000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=10000, fig=fig, ax=ax)

In [ ]:
particle = 3

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x1"][particle,:], boris["test_electrons"]["tracks"]["x2"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x1"][particle,:], gca["test_electrons"]["tracks"]["x2"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x1"][particle,:], boris["test_electrons"]["tracks"]["x2"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x1"][particle,:], gca["test_electrons"]["tracks"]["x2"][particle,:], label = "gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x1"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x2"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x2"]))

plt.show()


fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x1"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x1"][particle,:], label="gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["p2"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["p2"][particle,:], label="gca")

ax.set_title(r"$E \times B$ drift")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["p2"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["p2"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

## Gyromotion 3d

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Gyro_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Gyro_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"Gyromotion")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=10, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=10, fig=fig, ax=ax)

In [ ]:
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x1"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x1"][particle,:], label="gca")

ax.set_title(r"Gyromotion")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

## Curv 3d

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Curv_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Curv_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"Curvature")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

In [ ]:
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"Curvature")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

## $\nabla B$ Drift

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_GradB_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_GradB_Boris/Boris.in")

In [ ]:

pariticle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"$\nabla B$ Drift")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x1"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x1"][particle,:], label="gca")
ax.set_title(r"$\nabla B$ Drift")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()


fig, ax = plt.subplots()
# Linear fit for Boris p1 vs t
t_boris = boris["test_electrons"]["tracks"]["t"][particle, :]
p1_boris = boris["test_electrons"]["tracks"]["x1"][particle, :]

slope, intercept, r_value, p_value, std_err = linregress(t_boris, p1_boris)
fit_line = slope

ax.plot(t_boris, np.ones(boris["test_electrons"]["tracks"]["p1"][particle,:].shape) * fit_line, '--', label=f"Boris fit")
# ax.plot(t_boris, np.ones(boris["test_electrons"]["tracks"]["p1"][particle,:].shape) * 9.7087464904181361E-004 * 0.5 / 4.7, '--', label=f"Theoretical Value")
# ax.plot(t_boris, p1_boris, label="boris")

# ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], np.ones(boris["test_electrons"]["tracks"]["p1"][particle,:].shape) * np.average(boris["test_electrons"]["tracks"]["p1"][particle,:]), label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["p1"][particle,:], label="gca")

ax.set_title(r"$\nabla B$ Drift")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["p1"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["p1"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

## ( V_E . Grad ) b and parallel terms

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"( E . Grad ) b")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"( E . Grad ) b")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_v2_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_v2_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"( E . Grad ) b")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"( E . Grad ) b")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_v3_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_EGradB_v3_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"( E . Grad ) b")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"( E . Grad ) b")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

## Mirror

In [ ]:

def plot_field_line_through_point(
    x0, y0, z0,
    npts=800,          # number of samples along the curve
    z_span=10.0,       # plot from z0 - z_span to z0 + z_span
    fig=None,
    ax=None,
    B0=2.5, a=0.025, b=0.025
):
    """
    Plot the field line of B(x,y,z) = (-a*x*z, -a*y*z, B0 + b*z^2)
    passing through (x0, y0, z0).

    For this field, the field-line shape can be parameterized by z:
        x(z) = x0 * sqrt((B0 + b*z0^2)/(B0 + b*z^2))
        y(z) = y0 * sqrt((B0 + b*z0^2)/(B0 + b*z^2))
        z    = z
    Notes:
      * If x0=y0=0, the line is the z-axis.
      * The field is solenoidal for a=b (∇·B=0).
    """

    created_axes = False
    if fig is None or ax is None:
        fig = plt.figure(figsize=(10, 7)) if fig is None else fig
        ax = fig.add_subplot(111, projection='3d') if ax is None else ax
    else:
        old_xlim = ax.get_xlim3d()
        old_ylim = ax.get_ylim3d()
        old_zlim = ax.get_zlim3d()
        created_axes = True  # meaning: we received an existing axes with limits

    # z grid (both directions from z0)
    n_back = npts // 2
    n_fwd  = npts - n_back
    z_back = np.linspace(z0, z0 - z_span, n_back, endpoint=True)
    z_fwd  = np.linspace(z0, z0 + z_span, n_fwd,  endpoint=True)
    z = np.concatenate((z_back[::-1], z_fwd[1:]))  # avoid duplicating z0

    denom0 = B0 + b * z0**2
    denom  = B0 + b * z**2

    # avoid any numerical issues (denom is >0 for given B0,b, but be safe)
    eps = 1e-14
    denom = np.maximum(denom, eps)

    scale = np.sqrt(denom0 / denom)

    if (x0 == 0.0) and (y0 == 0.0):
        x = np.zeros_like(z)
        y = np.zeros_like(z)
    else:
        x = x0 * scale
        y = y0 * scale

    ax.plot3D(x, y, z, linewidth=1.0, linestyle="--", label="Field line", color = "black")

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.legend(loc="best")

    # If we were given an existing axes, restore its limits
    if created_axes:
        ax.set_xlim3d(old_xlim)
        ax.set_ylim3d(old_ylim)
        ax.set_zlim3d(old_zlim)

    return fig, ax


In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Mirror_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Mirror_Boris/Boris.in")

In [ ]:
particle = 0


fig, ax = plt.subplots()
# ax.plot(boris["test_electrons"]["tracks"]["t"][particle,1:], boris["test_electrons"]["tracks"]["p3"][particle,1:], label="boris - p3")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,1:], gca["test_electrons"]["tracks"]["p3"][particle,1:], label="gca - p3")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,1:], gca["test_electrons"]["tracks"]["p1"][particle,1:], label="gca - p1")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,1:], gca["test_electrons"]["tracks"]["p2"][particle,1:], label="gca - p2")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,1:], np.sqrt(gca["test_electrons"]["tracks"]["p1"][particle,1:]**2 + gca["test_electrons"]["tracks"]["p2"][particle,1:]**2 + gca["test_electrons"]["tracks"]["p3"][particle,1:]**2), label="gca - p")
# ax.plot(gca["test_electrons"]["tracks"]["t"][particle,1:], gca_parallel, label="gca - p_parallel")


ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("$p $" + "$[{}]$".format(boris["test_electrons"]["tracks"].units["p3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.grid(True)

# ax.set_xlim([24, 25])
plt.show()




fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_field_line_through_point(gca["test_electrons"]["tracks"]["x1"][0, 0], 
                                        gca["test_electrons"]["tracks"]["x2"][0, 0], 
                                        gca["test_electrons"]["tracks"]["x3"][0, 0], 
                                        fig=fig, ax=ax)





fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x2")
ax.set_ylabel("x3")

plt.show()




fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")


ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()

## Mixed drifts

In [ ]:
gca = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Mix_Gca/Gca.in")
boris = ou.Simulation("/home/exxxx5/Tese/Decks/GCA_PusherTest/3d_Mix_Boris/Boris.in")

In [ ]:
particle = 0
fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.scatter(boris["test_electrons"]["tracks"]["x2"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label = "boris")
ax.scatter(gca["test_electrons"]["tracks"]["x2"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label = "gca")

ax.set_xlabel("x1")
ax.set_ylabel("x2")

ax.set_title(r"( E . Grad ) b")


plt.show()

fig, ax = None, None
fig, ax = plot_trajectory(boris, r"Boris", particle=particle, tmax=1000, fig=fig, ax=ax)
fig, ax = plot_trajectory(gca, r"GCA", particle=particle, tmax=1000, fig=fig, ax=ax)

fig, ax = plt.subplots()
ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x3"][particle,:], label="boris")
ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x3"][particle,:], label="gca")

ax.set_title(r"( E . Grad ) b")
ax.set_xlabel("${}$".format(boris["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["t"]))
ax.set_ylabel("${}$".format(boris["test_electrons"]["tracks"].labels["x3"]) + "$[{}]$".format(boris["test_electrons"]["tracks"].units["x3"]))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# ax.set_xlim([24, 25])
plt.show()